### Representation - Commons votes

### Links

[UK Parliament Developer Hub](https://developer.parliament.uk/)

In [1]:
import pandas as pd
import requests
import urllib
import time

In [2]:
url_base = "https://members-api.parliament.uk/api"

## 1. Data

### 1.1 Members

In [3]:
# Members - Medway
path_members = 'data/representation/members.json'
df_members = (
    pd.read_json(path_members)
)

df_members

,links,value.id,value.name,value.startDate,value.endDate,value.currentRepresentation.member.value.id,value.currentRepresentation.member.value.nameListAs,value.currentRepresentation.member.value.nameDisplayAs,value.currentRepresentation.member.value.nameFullTitle,value.currentRepresentation.member.value.nameAddressAs,...,value.currentRepresentation.member.links,value.currentRepresentation.representation.membershipFrom,value.currentRepresentation.representation.membershipFromId,value.currentRepresentation.representation.house,value.currentRepresentation.representation.membershipStartDate,value.currentRepresentation.representation.membershipEndDate,value.currentRepresentation.representation.membershipEndReason,value.currentRepresentation.representation.membershipEndReasonNotes,value.currentRepresentation.representation.membershipEndReasonId,value.currentRepresentation.representation.membershipStatus
0,"[{'rel': 'self', 'href': '/Constituency/4262/R...",4262,Rochester and Strood,2024-05-31T00:00:00,NaN,5298,"Edwards, Lauren",Lauren Edwards,Lauren Edwards MP,NaN,...,"[{'rel': 'self', 'href': '/Members/5298', 'met...",Rochester and Strood,4262,1,2024-07-04T00:00:00,NaN,NaN,NaN,NaN,NaN
1,"[{'rel': 'self', 'href': '/Constituency/3972/C...",3972,Chatham and Aylesford,2024-05-31T00:00:00,NaN,5220,"Osborne, Tristan",Tristan Osborne,Tristan Osborne MP,NaN,...,"[{'rel': 'self', 'href': '/Members/5220', 'met...",Chatham and Aylesford,3972,1,2024-07-04T00:00:00,NaN,NaN,NaN,NaN,NaN
2,"[{'rel': 'self', 'href': '/Constituency/4061/G...",4061,Gillingham and Rainham,2024-05-31T00:00:00,NaN,5215,"Khan, Naushabah",Naushabah Khan,Naushabah Khan MP,NaN,...,"[{'rel': 'self', 'href': '/Members/5215', 'met...",Gillingham and Rainham,4061,1,2024-07-04T00:00:00,NaN,NaN,NaN,NaN,NaN


In [4]:
# Get member voting


def get_member_voting(member_id):    

    url = urllib.parse.urljoin(url_base, f"/api/Members/{member_id}/Voting")

    page = 1
    
    items = []
    while url:
        params = {
            'house': 1, 
            'page': page
        }
        print(f"fetching: {url, params}")
        response = requests.get(url, params=params)
        data = response.json()

        if len(data['items']) > 0:
            items.extend(data['items'])
            page+=1
        else:
            url = None
                        
        time.sleep(5)
    
    
    return items

for member_id in df_members['value.currentRepresentation.member.value.id']:
    print(member_id)
    member_voting = get_member_voting(member_id)
    df_member_votes = (
        pd.json_normalize(member_voting)
        .to_json(f"data/representation/member_commons_votes/{member_id}.json", orient='records', index=False)
    )



5298
fetching: ('https://members-api.parliament.uk/api/Members/5298/Voting', {'house': 1, 'page': 1})
fetching: ('https://members-api.parliament.uk/api/Members/5298/Voting', {'house': 1, 'page': 2})
fetching: ('https://members-api.parliament.uk/api/Members/5298/Voting', {'house': 1, 'page': 3})
fetching: ('https://members-api.parliament.uk/api/Members/5298/Voting', {'house': 1, 'page': 4})
5220
fetching: ('https://members-api.parliament.uk/api/Members/5220/Voting', {'house': 1, 'page': 1})
fetching: ('https://members-api.parliament.uk/api/Members/5220/Voting', {'house': 1, 'page': 2})
fetching: ('https://members-api.parliament.uk/api/Members/5220/Voting', {'house': 1, 'page': 3})
fetching: ('https://members-api.parliament.uk/api/Members/5220/Voting', {'house': 1, 'page': 4})
5215
fetching: ('https://members-api.parliament.uk/api/Members/5215/Voting', {'house': 1, 'page': 1})
fetching: ('https://members-api.parliament.uk/api/Members/5215/Voting', {'house': 1, 'page': 2})
fetching: ('htt

In [5]:
# Load voting data
df_members_voting = (
    pd.concat(
        [pd.read_json(f"data/representation/member_commons_votes/{member_id}.json").assign(**{'member_id': member_id}) 
        for member_id in df_members['value.currentRepresentation.member.value.id']]
    )
    .merge(df_members, how='right', left_on='member_id', right_on='value.currentRepresentation.member.value.id')
    .rename(columns={
        'value.currentRepresentation.member.value.nameDisplayAs': 'member_name', 
        'value.id_x': 'voting_id', 
        'value.date': 'date_vote',
        'value.title': 'division_name',
        'value.divisionNumber': 'division_number'
    })
)
df_members_voting.columns

Index(['links_x', 'value.house', 'voting_id', 'value.inAffirmativeLobby',
       'value.actedAsTeller', 'division_name', 'date_vote', 'division_number',
       'value.numberInFavour', 'value.numberAgainst', 'member_id', 'links_y',
       'value.id_y', 'value.name', 'value.startDate', 'value.endDate',
       'value.currentRepresentation.member.value.id',
       'value.currentRepresentation.member.value.nameListAs', 'member_name',
       'value.currentRepresentation.member.value.nameFullTitle',
       'value.currentRepresentation.member.value.nameAddressAs',
       'value.currentRepresentation.member.value.latestParty.id',
       'value.currentRepresentation.member.value.latestParty.name',
       'value.currentRepresentation.member.value.latestParty.abbreviation',
       'value.currentRepresentation.member.value.latestParty.backgroundColour',
       'value.currentRepresentation.member.value.latestParty.foregroundColour',
       'value.currentRepresentation.member.value.latestParty.isLord

### 1.3 Divisions

In [6]:
# Final all divisions since a date
# https://www.gov.uk/government/news/state-opening-of-parliament-to-take-place-on-17-july-2024

def get_divisions_from_date(start_date='2024-07-09'):
    return_format = 'json'
    url = urllib.parse.urljoin("https://commonsvotes-api.parliament.uk/", f"/data/divisions.{return_format}/search")
    divisions = []
    skip = 0
    while url:
        params={
            'startDate': start_date,
            'skip': skip,
            'take': 25
        }
        print(f"Fetching {url} {params}")
        response = requests.get(url=url, params=params)
    
        if len(response.json()) > 0:
            divisions.extend(response.json())
            skip += 25
            time.sleep(2)
        else:
            url = None
    return divisions

divisions = get_divisions_from_date()


Fetching https://commonsvotes-api.parliament.uk/data/divisions.json/search {'startDate': '2024-07-09', 'skip': 0, 'take': 25}
Fetching https://commonsvotes-api.parliament.uk/data/divisions.json/search {'startDate': '2024-07-09', 'skip': 25, 'take': 25}
Fetching https://commonsvotes-api.parliament.uk/data/divisions.json/search {'startDate': '2024-07-09', 'skip': 50, 'take': 25}
Fetching https://commonsvotes-api.parliament.uk/data/divisions.json/search {'startDate': '2024-07-09', 'skip': 75, 'take': 25}


In [7]:
df_divisions = (
        pd.DataFrame.from_records(divisions)
    )

df_divisions

,DivisionId,Date,PublicationUpdated,Number,IsDeferred,EVELType,EVELCountry,Title,AyeCount,NoCount,...,DoubleMajorityNoCount,AyeTellers,NoTellers,Ayes,Noes,FriendlyDescription,FriendlyTitle,NoVoteRecorded,RemoteVotingStart,RemoteVotingEnd
0,1888,2024-12-10T16:27:00,2024-12-10T16:56:21,62,False,,,Draft Home Detention Curfew and Requisite and ...,424,106,...,None,"[{'MemberId': 4871, 'Name': 'Christian Wakefor...","[{'MemberId': 4813, 'Name': 'Richard Holden', ...",[],[],None,None,[],None,None
1,1887,2024-12-10T16:11:00,2024-12-10T16:29:05,61,False,,,Finance Bill Committee New Clause 3,184,359,...,None,"[{'MemberId': 4813, 'Name': 'Richard Holden', ...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
2,1886,2024-12-10T15:56:00,2024-12-10T16:14:18,60,False,,,Finance Bill Committee: New Clause 2,74,350,...,None,"[{'MemberId': 5090, 'Name': 'Tom Morrison', 'P...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
3,1885,2024-12-10T14:17:00,2024-12-10T14:34:38,59,False,,,Finance Bill Committee: New Clause 5,105,340,...,None,"[{'MemberId': 5297, 'Name': 'Rebecca Paul', 'P...","[{'MemberId': 4449, 'Name': 'Anna Turley', 'Pa...",[],[],None,None,[],None,None
4,1884,2024-12-09T20:38:00,2024-12-09T21:00:40,58,False,,,Terrorism (Protection of Premises) Bill: New C...,89,340,...,None,"[{'MemberId': 5297, 'Name': 'Rebecca Paul', 'P...","[{'MemberId': 4993, 'Name': 'Keir Mather', 'Pa...",[],[],None,None,[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1831,2024-07-25T13:08:00,2024-07-25T13:33:27,5,False,,,The draft Criminal Justice Act 2003 (Requisite...,323,81,...,None,"[{'MemberId': 4501, 'Name': 'Gerald Jones', 'P...","[{'MemberId': 4384, 'Name': 'Mike Wood', 'Part...",[],[],None,None,[],None,None
58,1830,2024-07-23T19:30:00,2024-07-24T19:12:05,4,False,,,King's Speech (Motion for an Address): Amendme...,85,382,...,None,"[{'MemberId': 4612, 'Name': 'Jamie Stone', 'Pa...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
59,1829,2024-07-23T19:15:00,2024-07-23T20:10:34,3,False,,,King's Speech (Motion for an Address): Amendme...,103,363,...,None,"[{'MemberId': 4357, 'Name': 'Kirsty Blackman',...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
60,1828,2024-07-23T18:59:00,2024-07-24T11:30:21,2,False,,,King's Speech (Motion for an Address): Amendme...,117,384,...,None,"[{'MemberId': 4759, 'Name': 'Joy Morrissey', '...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None


### 3.1 Alignment

In [8]:

def _color_red_or_green(val):
    if val is None:
        color='blue'
    elif val == True:
        color = 'green'
    else:
        color = 'red'
    return 'color: %s' % color


(
    df_members_voting[['date_vote','member_id','member_name','voting_id', 'value.divisionNumber', 'value.inAffirmativeLobby',
       'division_name', 'value.divisionNumber', 'value.numberInFavour', 'value.numberAgainst'
    ]]
    
    .reset_index()
    .assign(**{
        'date_vote': lambda _df: pd.to_datetime(_df['date_vote']).dt.date
    })
    .pivot(index=["date_vote", "Vote"], columns=["member_name"], values=["value.inAffirmativeLobby"])
    # .pipe(lambda _df: _df.columns.get_level_values(1))
    .pipe(lambda _df: _df.set_axis( [col[1] for col in _df.columns.values], axis=1))
    # .reset_index()
    .assign(**{
        'aligned': lambda _df: _df.apply(lambda _row: _row['Lauren Edwards'] == _row['Naushabah Khan'] == _row['Tristan Osborne'], axis=1)
    })
    .loc[lambda _df: _df['aligned'] == False]
    .fillna("")
    .sort_values('date_vote', ascending=False)
).style.map(_color_red_or_green)

KeyError: "['value.divisionNumber'] not in index"

In [9]:
# Which divisions did our local politicians not vote in?
voted_in = df_members_voting['voting_id'].unique()

df_divisions.loc[lambda _df: _df['DivisionId'].isin(voted_in)]

,DivisionId,Date,PublicationUpdated,Number,IsDeferred,EVELType,EVELCountry,Title,AyeCount,NoCount,...,DoubleMajorityNoCount,AyeTellers,NoTellers,Ayes,Noes,FriendlyDescription,FriendlyTitle,NoVoteRecorded,RemoteVotingStart,RemoteVotingEnd
0,1888,2024-12-10T16:27:00,2024-12-10T16:56:21,62,False,,,Draft Home Detention Curfew and Requisite and ...,424,106,...,None,"[{'MemberId': 4871, 'Name': 'Christian Wakefor...","[{'MemberId': 4813, 'Name': 'Richard Holden', ...",[],[],None,None,[],None,None
1,1887,2024-12-10T16:11:00,2024-12-10T16:29:05,61,False,,,Finance Bill Committee New Clause 3,184,359,...,None,"[{'MemberId': 4813, 'Name': 'Richard Holden', ...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
2,1886,2024-12-10T15:56:00,2024-12-10T16:14:18,60,False,,,Finance Bill Committee: New Clause 2,74,350,...,None,"[{'MemberId': 5090, 'Name': 'Tom Morrison', 'P...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
3,1885,2024-12-10T14:17:00,2024-12-10T14:34:38,59,False,,,Finance Bill Committee: New Clause 5,105,340,...,None,"[{'MemberId': 5297, 'Name': 'Rebecca Paul', 'P...","[{'MemberId': 4449, 'Name': 'Anna Turley', 'Pa...",[],[],None,None,[],None,None
4,1884,2024-12-09T20:38:00,2024-12-09T21:00:40,58,False,,,Terrorism (Protection of Premises) Bill: New C...,89,340,...,None,"[{'MemberId': 5297, 'Name': 'Rebecca Paul', 'P...","[{'MemberId': 4993, 'Name': 'Keir Mather', 'Pa...",[],[],None,None,[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1831,2024-07-25T13:08:00,2024-07-25T13:33:27,5,False,,,The draft Criminal Justice Act 2003 (Requisite...,323,81,...,None,"[{'MemberId': 4501, 'Name': 'Gerald Jones', 'P...","[{'MemberId': 4384, 'Name': 'Mike Wood', 'Part...",[],[],None,None,[],None,None
58,1830,2024-07-23T19:30:00,2024-07-24T19:12:05,4,False,,,King's Speech (Motion for an Address): Amendme...,85,382,...,None,"[{'MemberId': 4612, 'Name': 'Jamie Stone', 'Pa...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
59,1829,2024-07-23T19:15:00,2024-07-23T20:10:34,3,False,,,King's Speech (Motion for an Address): Amendme...,103,363,...,None,"[{'MemberId': 4357, 'Name': 'Kirsty Blackman',...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None
60,1828,2024-07-23T18:59:00,2024-07-24T11:30:21,2,False,,,King's Speech (Motion for an Address): Amendme...,117,384,...,None,"[{'MemberId': 4759, 'Name': 'Joy Morrissey', '...","[{'MemberId': 4871, 'Name': 'Christian Wakefor...",[],[],None,None,[],None,None


In [ ]:
### Winning team
df_members_voting